In [11]:
from Bio import SeqIO
from Bio import Entrez
from sklearn.cluster import KMeans
from shutil import copyfile
from urllib.request import urlopen
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.cluster import hierarchy
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.pairwise import nan_euclidean_distances
from sklearn.ensemble import RandomForestClassifier as rf
import os
import glob
import time
import re
import pickle
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys

In [2]:
meta=pd.read_csv("../results/Plots/Metadata/meta_str_admix_K2.csv", index_col=0)
meta

,Species_STR2,POP,Procedencia,Latitude,Longitude,CRS,ADMIX_Cluster1,ADMIX_Cluster2,STR_Cluster1,STR_Cluster2
samples,,,,,,,,,,
CPB114sample,hibrido,-,cativeiro,NaN,NaN,NaN,0.255415,0.744585,0.258,0.742
CPB115sample,hibrido,-,cativeiro,NaN,NaN,NaN,0.241161,0.758839,0.244,0.756
CPB128sample,libidinosus,lib_cat,cativeiro,NaN,NaN,NaN,0.999990,0.000010,0.999,0.001
CPB129sample,flavius,CdI,cativeiro,NaN,NaN,NaN,0.000010,0.999990,0.000,1.000
CPB132sample,flavius,Mixed,cativeiro,NaN,NaN,NaN,0.101914,0.898086,0.105,0.895
...,...,...,...,...,...,...,...,...,...,...
DT03sample,libidinosus,lib_delta,livre,-2.804028,-41.866389,EPSG:4674,0.999990,0.000010,1.000,0.000
DT06sample,libidinosus,lib_delta,livre,-2.846389,-41.830556,EPSG:4674,0.999990,0.000010,1.000,0.000
DT8sample,libidinosus,-,livre,-7.929078,-44.195986,EPSG:4674,0.999990,0.000010,1.000,0.000


In [5]:
gen=pd.read_csv("../results/ML/filt_biallelic_filtered.genotype.csv", index_col=0)
gen

,loc10094_pos51,loc10104_pos11,loc10269_pos49,loc10436_pos97,loc10512_pos9,loc10623_pos107,loc10640_pos19,loc10663_pos5,loc10751_pos171,loc10789_pos175,...,loc9256_pos190,loc9292_pos20,loc9394_pos231,loc9407_pos55,loc9481_pos181,loc9500_pos93,loc9596_pos16,loc9781_pos170,loc9911_pos133,loc9991_pos107
CPB114sample,2,1,2,1,0,0,1,0,2,2,...,1,2,0,2,2,2,2,2,1,1
CPB115sample,1,1,1,1,1,1,2,0,2,2,...,2,2,1,1,2,2,2,2,2,2
CPB128sample,2,1,1,2,2,2,1,1,2,2,...,1,2,1,1,2,2,2,2,2,2
CPB129sample,0,0,1,2,2,0,2,0,2,2,...,2,2,0,2,2,2,2,2,2,2
CPB132sample,1,0,2,2,2,0,2,0,2,2,...,2,2,0,2,1,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DT03sample,1,2,1,2,2,2,2,2,0,2,...,1,2,2,0,2,2,2,2,2,2
DT06sample,2,2,2,0,2,2,2,2,1,1,...,2,2,2,2,2,2,2,2,2,1
DT8sample,1,2,2,2,2,1,2,2,1,1,...,0,2,2,2,2,2,2,0,2,1
MN37431sample,1,0,2,2,2,2,2,1,2,1,...,0,1,1,0,2,2,2,0,1,1
